In [ ]:
from yugiquery import *

init_notebook_mode(all_interactive=True)

header("My Collection")

---

Table of Contents
=================

*   [1  Data loading](#Data-loading)
    *   [1.1  Read collection](#Read-collection)
*   [2  Check changes](#Check-changes)
    *   [2.1  Load previous data](#Load-previous-data)
    *   [2.2  Generate changelog](#Generate-changelog)
    *   [2.3  Save data](#Save-data)
*   [3  Data visualization](#Data-visualization)
    *   [3.1  Full data](#Full-data)
    *   [3.2  Card types](#Card-types)
    *   [3.3  Monsters](#Monsters)
        *   [3.3.1  Attributes](#Attributes)
        *   [3.3.2  Primary types](#Primary-types)
            *   [3.3.2.1  Has effect discrimination](#Has-effect-discrimination)
            *   [3.3.2.2  Is pendulum discrimination](#Is-pendulum-discrimination)
            *   [3.3.2.3  By attribute](#By-attribute)
        *   [3.3.3  Secondary types](#Secondary-types)
            *   [3.3.3.1  By attribute](#By-attribute)
            *   [3.3.3.2  By secondary type](#By-secondary-type)
        *   [3.3.4  Monster types](#Monster-types)
            *   [3.3.4.1  By Attribute](#By-Attribute)
            *   [3.3.4.2  By primary type](#By-primary-type)
            *   [3.3.4.3  By secondary type](#By-secondary-type)
        *   [3.3.5  ATK](#ATK)
        *   [3.3.6  DEF](#DEF)
        *   [3.3.7  Level/Rank](#Level/Rank)
            *   [3.3.7.1  ATK statistics](#ATK-statistics)
            *   [3.3.7.2  DEF statistics](#DEF-statistics)
        *   [3.3.8  Pendulum scale](#Pendulum-scale)
            *   [3.3.8.1  ATK statistics](#ATK-statistics)
            *   [3.3.8.2  DEF statistics](#DEF-statistics)
            *   [3.3.8.3  Level/Rank statistics](#Level/Rank-statistics)
        *   [3.3.9  Link](#Link)
            *   [3.3.9.1  ATK statistics](#ATK-statistics)
        *   [3.3.10  Link Arrows](#Link-Arrows)
            *   [3.3.10.1  By combination](#By-combination)
            *   [3.3.10.2  By unique](#By-unique)
            *   [3.3.10.3  By link](#By-link)
    *   [3.4  Spell & Trap](#Spell-&-Trap)
        *   [3.4.1  Properties](#Properties)
    *   [3.5  Effect type](#Effect-type)
        *   [3.5.1  Card type discrimination](#Card-type-discrimination)
    *   [3.6  Archseries](#Archseries)
        *   [3.6.1  By card type](#By-card-type)
        *   [3.6.2  By primary type](#By-primary-type)
        *   [3.6.3  By secondary type](#By-secondary-type)
        *   [3.6.4  By monster type](#By-monster-type)
        *   [3.6.5  By property](#By-property)
    *   [3.7  Artworks](#Artworks)
        *   [3.7.1  By card type](#By-card-type)
        *   [3.7.2  By primary type](#By-primary-type)
    *   [3.8  Errata](#Errata)
        *   [3.8.1  By card type](#By-card-type)
        *   [3.8.2  By primary type](#By-primary-type)
        *   [3.8.3  By artwork](#By-artwork)
    *   [3.9  TCG & OCG status](#TCG-&-OCG-status)
        *   [3.9.1  TGC status](#TGC-status)
            *   [3.9.1.1  By card type](#By-card-type)
            *   [3.9.1.2  By monster type](#By-monster-type)
            *   [3.9.1.3  By archseries](#By-archseries)
        *   [3.9.2  OCG status](#OCG-status)
            *   [3.9.2.1  By card type](#By-card-type)
            *   [3.9.2.2  By monster type](#By-monster-type)
            *   [3.9.2.3  By archseries](#By-archseries)
        *   [3.9.3  TCG vs. OCG status](#TCG-vs.-OCG-status)
*   [4  Epilogue](#Epilogue)
    *   [4.1  HTML export](#HTML-export)
<!-- *   [4.2  Git](#Git) -->

# Data loading

## Read collection

In [ ]:
# Timestamp
timestamp = arrow.utcnow()

In [ ]:
git.ensure_repo()

In [ ]:
import requests


def read_ydk(file_path):
    with open(file_path, "r") as file:
        lines = file.readlines()

    data = []
    current_section = None

    for line in lines:
        line = line.strip()
        if not line:  # Skip empty lines
            continue
        if line in ["#main", "#extra", "!side"]:
            current_section = line[1:].capitalize()
        elif current_section:
            data.append({"Code": line, "Section": current_section, "Deck": file_path.stem})

    # Create a DataFrame
    df = pd.DataFrame(data)
    return df


def read_decklist(file_path):
    with open(file_path, "r") as file:
        lines = file.readlines()

    data = []
    current_section = None

    for line in lines:
        line = line.strip()
        if not line:  # Skip empty lines
            continue
        if line.endswith(":"):  # Section header
            current_section = line[:-1].capitalize()
        elif current_section:
            quantity, card_name = line.split("x ", 1)
            quantity = int(quantity)
            data.append({"Name": card_name, "Count": quantity, "Section": current_section, "Deck": file_path.stem})

    # Create a DataFrame
    df = pd.DataFrame(data)
    return df


def fetch_ygoprodeck():
    response = requests.get("https://db.ygoprodeck.com/api/v7/cardinfo.php")
    response.raise_for_status()
    result = response.json()
    return result["data"]


def convert_ydk(ydk_df):
    try:
        result = fetch_ygoprodeck()
        with open(dirs.DATA.joinpath("ygoprodeck.json"), "w") as file:
            json.dump(result, file, indent=4)
        ydk_data = pd.DataFrame(result).set_index("id")
    except Exception as e:
        print(e)
        ydk_data = pd.read_json(dirs.DATA.joinpath("ygoprodeck.json")).set_index("id")

    ydk_df = ydk_df.copy()

    def get_ydk_card(code):
        code = int(code)
        if code not in ydk_data.index:
            return np.nan
        return ydk_data.loc[code, "name"]

    ydk_df["Name"] = ydk_df["Code"].apply(get_ydk_card)
    not_found = ydk_df[ydk_df["Name"].isna()]["Code"].values
    if len(not_found) > 0:
        print(f"Could not find {len(not_found)} cards:")
        print(", ".join(not_found))
    ydk_df = ydk_df.drop("Code", axis=1).dropna(subset=["Name"]).reset_index(drop=True)
    ydk_df["Count"] = ydk_df.groupby(["Name", "Section", "Deck"])["Name"].transform("count")
    ydk_df = ydk_df.drop_duplicates().reset_index(drop=True)
    return ydk_df

In [ ]:
def get_ydk():
    ydk_files = list(dirs.DATA.glob("*.ydk"))
    ydk_df = pd.DataFrame()
    for file in ydk_files:
        temp_df = read_ydk(file)
        ydk_df = pd.concat([ydk_df, temp_df])
        print(f"Loaded {file.name} deck")

    ydk_df = convert_ydk(ydk_df)
    return ydk_df


def get_decklist():
    decklist_files = list(dirs.DATA.glob("*.txt"))
    decklist_df = pd.DataFrame()
    for file in decklist_files:
        temp_df = read_decklist(file)
        decklist_df = pd.concat([decklist_df, temp_df])
        print(f"Loaded {file.name} deck")

    decklist_df.replace({"Section": {"Monster": "Main", "Spell": "Main", "Trap": "Main"}}, inplace=True)
    return decklist_df

In [ ]:
# Get newest file if exist and correct tuples and timestamps
tuple_cols = [
    "Secondary type",
    "Effect type",
    "Link Arrows",
    "Archseries",
    "Artwork",
    "Errata",
    "Rarity",
    "Cover card",
]

list_df, _ = load_corrected_latest("cards", tuple_cols)

In [ ]:
decks_df = pd.concat([get_ydk(), get_decklist()])
decks_df = decks_df.merge(list_df, on="Name", how="left")

In [ ]:
decks_df

In [ ]:
collection_file = dirs.DATA.joinpath("collection.csv")
collection_df = pd.read_csv(collection_file, comment="#")
collection_df.dropna(how="all", inplace=True, axis=1)
print(f"Loaded {collection_file.name}")

In [ ]:
if "Card number" in collection_df.columns:
    set_lists_df, _ = load_corrected_latest("sets", tuple_cols)
    extra_names = (
        set_lists_df[["Name", "Card number"]]
        .drop_duplicates(subset=["Card number"])
        .merge(collection_df["Card number"].dropna(), on="Card number", how="right")
    )
    collection_df = pd.concat([collection_df.drop("Card number", axis=1), extra_names], ignore_index=True)

if "Password" in collection_df.columns:
    temp_df = list_df.drop_duplicates(subset=["Name"]).merge(collection_df["Name"].dropna(), on="Name", how="right")
    extra_entries = list_df[list_df["Password"].isin(collection_df["Password"].dropna())]
    collection_df = pd.concat([temp_df, extra_entries], ignore_index=True)
else:
    collection_df = list_df.drop_duplicates(subset=["Name"]).merge(collection_df["Name"].dropna(), on="Name", how="right")

In [ ]:
collection_df = pd.merge(left=collection_df, right=decks_df, how="outer")

In [ ]:
collection_df

# Check changes

## Load previous data

In [ ]:
# Get latest file if exist
tuple_cols = [
    "Effect type",
    "Secondary type",
    "Link Arrows",
    "Archseries",
    "Artwork",
    "Errata",
]
previous_df, previous_ts = load_corrected_latest("collection", tuple_cols)

if previous_df is not None:
    previous_df = previous_df.astype(collection_df[previous_df.columns.intersection(collection_df.columns)].dtypes.to_dict())

## Generate changelog

In [ ]:
if previous_df is None:
    changelog = None
    print("Skipped")
else:
    changelog = generate_changelog(previous_df, collection_df, col="Name")
    if not changelog.empty:
        display(changelog)
        changelog.to_csv(
            dirs.DATA
            / make_filename(
                report="collection",
                timestamp=timestamp,
                previous_timestamp=previous_ts,
            ),
            index=True,
        )
        print("Changelog saved")

## Save data

In [ ]:
if changelog is not None and changelog.empty:
    print("No changes. New data not saved")
else:
    collection_df.to_csv(
        dirs.DATA / make_filename(report="collection", timestamp=timestamp),
        index=False,
    )
    print("Data saved")

# Data visualization

## Full data

In [ ]:
collection_df

## Card types

In [ ]:
print("Total number of Card types:", collection_df["Card type"].nunique())

In [ ]:
card_type_colors = [plot.colors_dict[i] for i in collection_df["Card type"].value_counts().index]
collection_df["Card type"].value_counts().plot.bar(figsize=(18, 6), grid=True, rot=0, color=card_type_colors)
plt.show()

## Monsters

### Attributes

In [ ]:
print("Total number of attributes:", collection_df["Attribute"].nunique())

In [ ]:
collection_df.drop(columns=["Card type", "Page name", "Page URL"]).groupby("Attribute").nunique()

In [ ]:
attribute_colors = [plot.colors_dict[i] for i in collection_df["Attribute"].value_counts().index]
collection_df["Attribute"].value_counts().plot.bar(figsize=(18, 6), grid=True, rot=0, color=attribute_colors)
plt.show()

### Primary types

In [ ]:
print("Total number of primary types:", collection_df["Primary type"].nunique())

In [ ]:
collection_df.drop(columns=["Card type", "Page name", "Page URL"]).groupby("Primary type").nunique()

#### Has effect discrimination

In [ ]:
effect = pd.crosstab(
    collection_df["Primary type"],
    pd.isna(collection_df["Effect type"]),
    rownames=["Primary type"],
    colnames=["Has effect"],
).rename(columns={True: "No Effect", False: "Effect"})
effect

In [ ]:
monster_type_colors = {
    "No Effect": plot.colors_dict["Normal Monster"],
    "Effect": [plot.colors_dict[i] for i in effect.index],
}
effect.plot.bar(
    figsize=(18, 6),
    stacked=True,
    grid=True,
    rot=0,
    legend=True,
    color=monster_type_colors,
)
# plt.yscale('log')
plt.show()

Obs: Normal monster can have effect if it is pendulum

#### Is pendulum discrimination

In [ ]:
pendulum = pd.crosstab(
    collection_df["Primary type"],
    pd.isna(collection_df["Pendulum Scale"]),
    rownames=["Primary type"],
    colnames=["Is Pendulum"],
).rename(columns={True: "Not Pendulum", False: "Pendulum"})
pendulum

In [ ]:
monster_type_colors_b = {
    "Pendulum": plot.colors_dict["Pendulum Monster"],
    "Not Pendulum": [plot.colors_dict[i] for i in pendulum.index],
}
pendulum.plot.bar(
    figsize=(18, 6),
    stacked=True,
    grid=True,
    rot=0,
    color=monster_type_colors_b,
    legend=True,
    title="Primary types - Is pendulum",
)
plt.show()

#### By attribute

In [ ]:
primmary_crosstab = pd.crosstab(collection_df["Primary type"], collection_df["Attribute"])
primmary_crosstab

In [ ]:
plt.figure(figsize=(16, 10))
sns.heatmap(
    primmary_crosstab.T,
    annot=True,
    fmt="g",
    cmap="viridis",
    square=True,
    norm=plot.LogNorm(),
)
plt.show()

### Secondary types

In [ ]:
exploded_secondary_type = collection_df.explode("Secondary type")
print(
    "Total number of secondary types:",
    exploded_secondary_type["Secondary type"].nunique(),
)

In [ ]:
exploded_secondary_type.drop(columns=["Card type", "Link", "Link Arrows", "Page name", "Page URL"]).groupby(
    "Secondary type"
).nunique()

In [ ]:
secondary_type_colors = plot.colors_dict["Effect Monster"]
exploded_secondary_type["Secondary type"].value_counts().plot.bar(
    figsize=(18, 6),
    stacked=True,
    grid=True,
    rot=0,
    color=secondary_type_colors,
    legend=False,
)
plt.show()

#### By attribute

In [ ]:
secondary_crosstab = pd.crosstab(exploded_secondary_type["Secondary type"], exploded_secondary_type["Attribute"])
secondary_crosstab

In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(
    secondary_crosstab[secondary_crosstab > 0],
    annot=True,
    fmt="g",
    cmap="viridis",
    square=True,
)
plt.show()

#### By secondary type

In [ ]:
secondary_crosstab_b = pd.crosstab(
    exploded_secondary_type["Primary type"],
    exploded_secondary_type["Secondary type"],
    margins=True,
)
secondary_crosstab_b

In [ ]:
plt.figure(figsize=(10, 4))
sns.heatmap(
    secondary_crosstab_b[secondary_crosstab_b > 0],
    annot=True,
    fmt="g",
    cmap="viridis",
    square=True,
    # norm=plot.LogNorm(),
)
plt.show()

### Monster types

In [ ]:
print("Total number of monster types:", collection_df["Monster type"].nunique())

In [ ]:
collection_df.drop(columns=["Card type", "Page name", "Page URL"]).groupby("Monster type").nunique()

In [ ]:
monster_type_colors = plot.colors_dict["Monster Card"]
collection_df["Monster type"].value_counts().plot.bar(figsize=(18, 6), grid=True, rot=45, color=monster_type_colors)
plt.show()

#### By Attribute

In [ ]:
monster_crosstab = pd.crosstab(collection_df["Monster type"], collection_df["Attribute"])
monster_crosstab

In [ ]:
plt.figure(figsize=(20, 5))
sns.heatmap(
    monster_crosstab[monster_crosstab > 0].T,
    annot=True,
    fmt="g",
    cmap="viridis",
    square=True,
    # norm=plot.LogNorm(),
)
plt.show()

#### By primary type

In [ ]:
monster_crosstab_b = pd.crosstab(
    collection_df["Monster type"][collection_df["Monster type"].notna()],
    collection_df["Primary type"][collection_df["Monster type"].notna()],
    dropna=False,
)
monster_crosstab_b

In [ ]:
plt.figure(figsize=(20, 5))
sns.heatmap(
    monster_crosstab_b[monster_crosstab_b > 0].T,
    annot=True,
    fmt="g",
    cmap="viridis",
    square=True,
    norm=plot.LogNorm(),
)
plt.show()

#### By secondary type

In [ ]:
monster_crosstab_c = pd.crosstab(
    exploded_secondary_type["Monster type"][exploded_secondary_type["Monster type"].notna()],
    exploded_secondary_type["Secondary type"][exploded_secondary_type["Monster type"].notna()],
    dropna=False,
)
monster_crosstab_c

In [ ]:
plt.figure(figsize=(20, 5))
sns.heatmap(
    monster_crosstab_c[monster_crosstab_c > 0].T,
    annot=True,
    fmt="g",
    cmap="viridis",
    square=True,
    norm=plot.LogNorm(),
)
plt.show()

### ATK

In [ ]:
print("Total number of ATK values:", collection_df["ATK"].nunique())

In [ ]:
collection_df.drop(columns=["Card type", "Page name", "Page URL"]).groupby("ATK").nunique().sort_index(
    key=lambda x: pd.to_numeric(x, errors="coerce")
)

In [ ]:
atk_colors = plot.colors_dict["Monster Card"]
collection_df["ATK"].value_counts().sort_index(key=lambda x: pd.to_numeric(x, errors="coerce")).plot.bar(
    figsize=(18, 6), grid=True, color=atk_colors
)
plt.show()

### DEF

In [ ]:
print("Total number of DEF values:", collection_df["DEF"].nunique())

In [ ]:
collection_df.drop(columns=["Card type", "Page name", "Page URL"]).groupby("DEF").nunique().sort_index(
    key=lambda x: pd.to_numeric(x, errors="coerce")
)

In [ ]:
def_colors = plot.colors_dict["Monster Card"]
collection_df["DEF"].value_counts().sort_index(key=lambda x: pd.to_numeric(x, errors="coerce")).plot.bar(
    figsize=(18, 6), grid=True, color=def_colors
)
plt.show()

### Level/Rank

In [ ]:
collection_df.drop(columns=["Card type", "Link", "Link Arrows", "Page name", "Page URL"]).groupby(
    "Level/Rank"
).nunique().sort_index(key=lambda x: pd.to_numeric(x, errors="coerce"))

In [ ]:
stars_colors = plot.colors_dict["Level"]
collection_df["Level/Rank"].value_counts().sort_index(key=lambda x: pd.to_numeric(x, errors="coerce")).plot.bar(
    figsize=(18, 6), grid=True, rot=0, color=stars_colors
)
plt.show()

#### ATK statistics

In [ ]:
collection_df[["Level/Rank", "ATK"]].apply(pd.to_numeric, errors="coerce").dropna().astype(int).groupby(
    "Level/Rank"
).describe().round(1)

#### DEF statistics

In [ ]:
collection_df[["Level/Rank", "DEF"]].apply(pd.to_numeric, errors="coerce").dropna().astype(int).groupby(
    "Level/Rank"
).describe().round(1)

### Pendulum scale

In [ ]:
collection_df.drop(columns=["Card type", "Link", "Link Arrows", "Page name", "Page URL"]).groupby(
    "Pendulum Scale"
).nunique().sort_index(key=lambda x: pd.to_numeric(x, errors="coerce"))

In [ ]:
scales_colors = plot.colors_dict["Pendulum Monster"]
collection_df["Pendulum Scale"].value_counts().sort_index(key=lambda x: pd.to_numeric(x, errors="coerce")).plot.bar(
    figsize=(18, 6), grid=True, rot=0, color=scales_colors
)
plt.show()

#### ATK statistics

In [ ]:
collection_df[["Pendulum Scale", "ATK"]].apply(pd.to_numeric, errors="coerce").dropna().astype(int).groupby(
    "Pendulum Scale"
).describe().round(1)

#### DEF statistics

In [ ]:
collection_df[["Pendulum Scale", "DEF"]].apply(pd.to_numeric, errors="coerce").dropna().astype(int).groupby(
    "Pendulum Scale"
).describe().round(1)

#### Level/Rank statistics

In [ ]:
collection_df[["Pendulum Scale", "Level/Rank"]].apply(pd.to_numeric, errors="coerce").dropna().astype(int).groupby(
    "Pendulum Scale"
).describe().round(1)

### Link

In [ ]:
collection_df.drop(
    columns=[
        "Card type",
        "Primary type",
        "Secondary type",
        "Level/Rank",
        "DEF",
        "Pendulum Scale",
        "Page name",
        "Page URL",
    ]
).groupby("Link").nunique().sort_index(key=lambda x: pd.to_numeric(x, errors="coerce"))

In [ ]:
link_colors = plot.colors_dict["Link Monster"]
collection_df["Link"].value_counts().sort_index(key=lambda x: pd.to_numeric(x, errors="coerce")).plot.bar(
    figsize=(18, 6), grid=True, rot=0, color=link_colors
)
plt.show()

#### ATK statistics

In [ ]:
collection_df[["Link", "ATK"]].apply(pd.to_numeric, errors="coerce").dropna().astype(int).groupby("Link").describe().round(1)

### Link Arrows

#### By combination

In [ ]:
print("Total number of link arrow combinations:", collection_df["Link Arrows"].nunique())

In [ ]:
collection_df.drop(
    columns=[
        "Card type",
        "Primary type",
        "Level/Rank",
        "Pendulum Scale",
        "Link",
        "Secondary type",
        "DEF",
        "Page name",
        "Page URL",
    ]
).groupby("Link Arrows").nunique()

In [ ]:
arrows_colors = plot.colors_dict["Link Monster"]
collection_df["Link Arrows"].value_counts().plot.barh(
    figsize=(10, 20), grid=True, color=arrows_colors, title="Link arrows combinations"
)
plt.show()

#### By unique

In [ ]:
collection_df[collection_df["Link Arrows"].notna()].drop(
    columns=[
        "Card type",
        "Primary type",
        "Level/Rank",
        "Pendulum Scale",
        "Secondary type",
        "DEF",
        "Page name",
        "Page URL",
    ]
).explode("Link Arrows").groupby("Link Arrows").nunique()

In [ ]:
plot.arrows(collection_df["Link Arrows"].explode("Link Arrows"))

#### By link

In [ ]:
arrow_per_link = collection_df[["Link Arrows", "Link"]].explode("Link Arrows").dropna()
arrow_crosstab = pd.crosstab(arrow_per_link["Link Arrows"], arrow_per_link["Link"])
arrow_crosstab

In [ ]:
plt.figure(figsize=(10, 6))
sns.heatmap(
    arrow_crosstab[arrow_crosstab > 0].T,
    annot=True,
    fmt="g",
    cmap="viridis",
    square=True,
    # norm=plot.LogNorm(),
)
plt.show()

## Spell & Trap

### Properties

In [ ]:
print("Total number of properties:", collection_df["Property"].nunique())

In [ ]:
collection_df.drop(columns=["Card type", "Page name", "Page URL"]).groupby("Property").nunique()

In [ ]:
st_colors = [plot.colors_dict[i] for i in collection_df[["Card type", "Property"]].value_counts().index.get_level_values(0)]
collection_df["Property"].value_counts().plot.bar(figsize=(18, 6), grid=True, rot=45, color=st_colors)
plt.show()

## Effect type

In [ ]:
print("Total number of effect types:", collection_df["Effect type"].explode().nunique())

In [ ]:
collection_df.explode("Effect type").groupby("Effect type").nunique()

### Card type discrimination

In [ ]:
st_diff = collection_df[["Card type", "Effect type"]].explode("Effect type").value_counts().unstack(0).fillna(0).astype(int)
st_diff

In [ ]:
st_diff_colors = {
    "Monster Card": plot.colors_dict["Monster Card"],
    "Spell Card": plot.colors_dict["Spell Card"],
    "Trap Card": plot.colors_dict["Trap Card"],
}
st_diff.plot.bar(figsize=(18, 6), stacked=True, grid=True, rot=45, color=st_diff_colors)
plt.show()

## Archseries

In [ ]:
exploded_archseries = collection_df.explode("Archseries")
print("Total number of Archseries:", exploded_archseries["Archseries"].nunique())

In [ ]:
exploded_archseries.groupby("Archseries").nunique()

In [ ]:
exploded_archseries["Archseries"].value_counts().plot.barh(figsize=(10, 100), grid=True, title="Archtypes/Series")
plt.show()

### By card type

In [ ]:
archseries_crosstab = pd.crosstab(exploded_archseries["Archseries"], exploded_archseries["Card type"], margins=True)
archseries_crosstab

### By primary type

In [ ]:
archseries_crosstab_b = pd.crosstab(exploded_archseries["Archseries"], exploded_archseries["Primary type"], margins=True)
archseries_crosstab_b



### By secondary type

In [ ]:
exploded_archseries_secondary_type = exploded_archseries[["Archseries", "Secondary type"]].explode("Secondary type")
archseries_crosstab_c = pd.crosstab(
    exploded_archseries_secondary_type["Archseries"],
    exploded_archseries_secondary_type["Secondary type"],
    margins=True,
)
archseries_crosstab_c

### By monster type

In [ ]:
archseries_crosstab_d = pd.crosstab(exploded_archseries["Archseries"], exploded_archseries["Monster type"], margins=True)
archseries_crosstab_d

### By property

In [ ]:
archseries_crosstab_e = pd.crosstab(exploded_archseries["Archseries"], exploded_archseries["Property"], margins=True)
archseries_crosstab_e

## Artworks

In [ ]:
print(
    "Total number of cards with edited or alternate artworks:",
    collection_df["Artwork"].count(),
)

In [ ]:
collection_df[["Name", "Password", "TCG status", "OCG status", "Artwork"]][collection_df["Artwork"].notna()]

In [ ]:
artwork_value_counts = collection_df["Artwork"].value_counts()
plt.figure(figsize=(20, 8))
plt.title("Artworks")
plot.venn2(
    subsets=(
        artwork_value_counts[("Alternate",)],
        artwork_value_counts[("Edited",)],
        artwork_value_counts[("Alternate", "Edited")],
    ),
    set_labels=("Alternate artwork", "Edited artwork"),
)
plt.show()

### By card type

In [ ]:
artwork_crosstab = pd.crosstab(collection_df["Artwork"], collection_df["Card type"])
artwork_crosstab

### By primary type

In [ ]:
artwork_crosstab_b = pd.crosstab(collection_df["Artwork"], collection_df["Primary type"])
artwork_crosstab_b

More granularity is unnecessary

## Errata

In [ ]:
print("Total number of cards with errata:", collection_df["Errata"].count())

In [ ]:
collection_df[["Name", "Password", "TCG status", "OCG status", "Errata"]][collection_df["Errata"].notna()]

In [ ]:
errata_counts = collection_df.groupby("Errata").nunique().sort_values("Name", ascending=False)
errata_counts

In [ ]:
plt.figure(figsize=(20, 8))
plt.title("Errata")
sorted_errata_name_counts = errata_counts["Name"].drop(("Any",)).sort_index(key=lambda x: [(len(i), i) for i in x])
plot.venn2(
    subsets=sorted_errata_name_counts,
    set_labels=sorted_errata_name_counts.index[:-1].str[0],
)
plt.show()

### By card type

In [ ]:
errata_crosstab = pd.crosstab(collection_df["Errata"], collection_df["Card type"])
errata_crosstab.sort_values(by=errata_crosstab.columns.tolist(), ascending=False)

### By primary type

In [ ]:
errata_crosstab_b = pd.crosstab(collection_df["Errata"], collection_df["Primary type"])
errata_crosstab_b.sort_values(by=errata_crosstab_b.columns.tolist(), ascending=False)

More granularity is unnecessary

### By artwork

In [ ]:
errata_crosstab_c = pd.crosstab(collection_df["Artwork"], collection_df["Errata"])
errata_crosstab_c.sort_values(by=errata_crosstab_c.columns.tolist(), ascending=False)

## TCG & OCG status

### TGC status

In [ ]:
print("Total number of TCG status:", collection_df["TCG status"].nunique())

In [ ]:
collection_df.drop(columns=["Page name", "Page URL"]).groupby("TCG status", dropna=False).nunique()

In [ ]:
collection_df["TCG status"].value_counts(dropna=False).plot.bar(figsize=(18, 6), logy=True, grid=True, rot=45)
plt.show()

#### By card type

In [ ]:
# Remove unlimited
tcg_crosstab = pd.crosstab(collection_df["Card type"], collection_df["TCG status"]).drop(["Unlimited"], axis=1)
tcg_crosstab

In [ ]:
plt.figure(figsize=(12, 6))
sns.heatmap(
    tcg_crosstab[tcg_crosstab > 0],
    annot=True,
    fmt="g",
    cmap="viridis",
    # norm=plot.LogNorm(),
)
plt.show()

#### By monster type

In [ ]:
# Remove unlimited
tcg_crosstab_b = pd.crosstab(collection_df["Monster type"], collection_df["TCG status"]).drop(["Unlimited"], axis=1)
tcg_crosstab_b

In [ ]:
plt.figure(figsize=(20, 4))
sns.heatmap(
    tcg_crosstab_b[tcg_crosstab_b > 0].T.dropna(how="all", axis=1),
    annot=True,
    fmt="g",
    cmap="viridis",
    square=True,
)
plt.show()

#### By archseries

In [ ]:
# Remove unlimited
tcg_crosstab_c = pd.crosstab(
    exploded_archseries["Archseries"].where(exploded_archseries["OCG status"] != "Unlimited"),
    exploded_archseries["TCG status"],
    margins=True,
)
tcg_crosstab_c

### OCG status

In [ ]:
print("Total number of OCG status:", collection_df["OCG status"].nunique())

In [ ]:
collection_df.drop(columns=["Page name", "Page URL"]).groupby("OCG status", dropna=False).nunique()

In [ ]:
collection_df["OCG status"].value_counts(dropna=False).plot.bar(figsize=(18, 6), logy=True, grid=True, rot=45)
plt.show()

#### By card type

In [ ]:
# Remove unlimited
ocg_crosstab = pd.crosstab(collection_df["Card type"], collection_df["OCG status"]).drop(["Unlimited"], axis=1)
ocg_crosstab

In [ ]:
plt.figure(figsize=(12, 6))
sns.heatmap(ocg_crosstab[ocg_crosstab > 0], annot=True, fmt="g", cmap="viridis")
plt.show()

#### By monster type

In [ ]:
ocg_crosstab_b = pd.crosstab(collection_df["Monster type"], collection_df["OCG status"]).drop(["Unlimited"], axis=1)
ocg_crosstab_b

In [ ]:
plt.figure(figsize=(20, 4))
sns.heatmap(
    ocg_crosstab_b[ocg_crosstab_b > 0].T.dropna(how="all", axis=1),
    annot=True,
    fmt="g",
    cmap="viridis",
    square=True,
)
plt.show()

#### By archseries

In [ ]:
# Remove unlimited
ocg_crosstab_c = pd.crosstab(
    exploded_archseries["Archseries"].where(exploded_archseries["OCG status"] != "Unlimited"),
    exploded_archseries["OCG status"],
    margins=True,
)
ocg_crosstab_c

### TCG vs. OCG status

In [ ]:
cg_crosstab = pd.crosstab(
    collection_df["OCG status"],
    collection_df["TCG status"],
    dropna=False,
    margins=False,
)
cg_crosstab

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(
    cg_crosstab[cg_crosstab > 0],
    annot=True,
    fmt="g",
    cmap="viridis",
    square=True,
    norm=plot.LogNorm(),
)
plt.show()

# Epilogue

In [ ]:
# benchmark(report='Collection',timestamp=timestamp)

In [ ]:
footer(timestamp)

## HTML export

In [ ]:
# Save notebook on disck before generating HTML report
save_notebook()

In [ ]:
export_notebook(dirs.NOTEBOOKS.user / "Collection.ipynb")

## Git

In [ ]:
git.commit("*[Cc]olection*", f"Collection update - {timestamp.isoformat()}")